In [1]:
import requests
import json
import re
from typing import List, Tuple

In [2]:
class FLAREDirect:
    def __init__(self, ollama_url: str = "http://localhost:11434"):
        self.ollama_url = ollama_url
        self.model = "llama3"
    
    def call_llama(self, prompt: str, max_tokens: int = 200) -> str:
        url = f"{self.ollama_url}/api/generate"
        data = {
            "model": self.model,
            "prompt": prompt,
            "stream": False,
            "options": {
                "num_predict": max_tokens,
                "temperature": 0.7
            }
        }
        
        try:
            response = requests.post(url, json=data, timeout=30)
            response.raise_for_status()
            return response.json()["response"].strip()
        except Exception as e:
            return f"Error calling Llama: {str(e)}"
    
    def search_web(self, query: str) -> str:
        # In practice, you'd use a real search API or knowledge base
        
        retrieval_db = {
            "kabhie kabhie cast": "Kabhie Kabhie starred Amitabh Bachchan, Shashi Kapoor, Rakhee Gulzar, Waheeda Rehman, Rishi Kapoor, and Neetu Singh in lead roles.",
            "kabhie kabhie director": "Kabhie Kabhie was directed by Yash Chopra and produced by Yash Raj Films.",
            "kabhie kabhie release": "Kabhie Kabhie was released on 27 February 1976 in India.",
            "kabhie kabhie music": "The music for Kabhie Kabhie was composed by Khayyam with lyrics by Sahir Ludhianvi.",
            "kabhie kabhie plot": "The film tells the story of a poet Amit who falls in love with Pooja, but circumstances separate them. Years later, their paths cross again in unexpected ways.",
            "kabhie kabhie box office": "Kabhie Kabhie was a commercial success and became one of the highest-grossing films of 1976."
        }
        
        query_lower = query.lower()
        for key, value in retrieval_db.items():
            if any(word in query_lower for word in key.split()):
                return value
        
        return "No relevant information found."
    
    def detect_uncertainty(self, text: str) -> List[str]:
        uncertainty_patterns = [
            r"I (?:think|believe|assume|guess|suppose)",
            r"(?:probably|possibly|maybe|perhaps|likely)",
            r"I'm not (?:sure|certain)",
            r"(?:unclear|unknown|uncertain)",
            r"\[.*?\]",  # Placeholder patterns
            r"need to (?:check|verify|confirm)",
        ]
        
        uncertain_phrases = []
        for pattern in uncertainty_patterns:
            matches = re.findall(pattern, text, re.IGNORECASE)
            uncertain_phrases.extend(matches)
        
        return uncertain_phrases
    
    def extract_search_queries(self, text: str, uncertain_phrases: List[str]) -> List[str]:
        queries = []
        
        # Look for specific information needs
        if any("cast" in phrase.lower() for phrase in uncertain_phrases):
            queries.append("kabhie kabhie cast")
        if any("director" in phrase.lower() for phrase in uncertain_phrases):
            queries.append("kabhie kabhie director")
        if any("release" in phrase.lower() for phrase in uncertain_phrases):
            queries.append("kabhie kabhie release")
        if any("music" in phrase.lower() for phrase in uncertain_phrases):
            queries.append("kabhie kabhie music")
        
        # Fallback: use the movie name with context
        if not queries:
            queries.append("kabhie kabhie bollywood movie details")
        
        return queries
    
    def generate_wikipedia_page(self, movie_title: str, max_iterations: int = 5) -> str:
        
        initial_prompt = f"""Write a Wikipedia-style article about the Bollywood movie "{movie_title}".
                            Include sections like Plot, Cast, Production, Music, Reception, etc.
                            Write in encyclopedic style with factual information.
                            If you're uncertain about specific details, indicate that uncertainty.

                            {movie_title}:
                            """
        
        current_text = ""
        iteration = 0
        
        print(f"Starting FLARE-Direct generation for '{movie_title}'...")
        print("=" * 60)
        
        while iteration < max_iterations:
            iteration += 1
            print(f"\n--- Iteration {iteration} ---")
            
            # Generate next chunk
            prompt = initial_prompt + current_text
            generated_chunk = self.call_llama(prompt, max_tokens=150)
            
            print(f"Generated: {generated_chunk[:100]}...")
            
            # Detect uncertainty
            uncertain_phrases = self.detect_uncertainty(generated_chunk)
            
            if uncertain_phrases:
                print(f"Uncertainty detected: {uncertain_phrases}")
                
                # Extract search queries
                search_queries = self.extract_search_queries(generated_chunk, uncertain_phrases)
                
                # Retrieve information
                retrieved_info = []
                for query in search_queries:
                    print(f"Searching for: {query}")
                    info = self.search_web(query)
                    retrieved_info.append(info)
                    print(f"Retrieved: {info[:80]}...")
                
                # Incorporate retrieved information
                if retrieved_info:
                    retrieval_context = "\n".join(retrieved_info)
                    refinement_prompt = f"""Based on this additional information:
                                            {retrieval_context}

                                            Rewrite and improve this text:
                                            {generated_chunk}

                                            Make it more factual and remove uncertainty markers:
                                            """
                    refined_chunk = self.call_llama(refinement_prompt, max_tokens=200)
                    current_text += refined_chunk + "\n\n"
                    print(f"Refined with retrieval: {refined_chunk[:100]}...")
                else:
                    current_text += generated_chunk + "\n\n"
            else:
                current_text += generated_chunk + "\n\n"
                print("No uncertainty detected, continuing generation...")
            
            # Stop if we have enough content
            if len(current_text) > 1000:
                break
        
        return current_text.strip()

In [3]:
if __name__ == "__main__":
    # Initialize FLARE-Direct
    flare = FLAREDirect()
    
    # Generate Wikipedia page
    wiki_page = flare.generate_wikipedia_page("Kabhie Kabhie")
    
    print("\n" + "=" * 60)
    print("FINAL WIKIPEDIA PAGE:")
    print("=" * 60)
    print(wiki_page)
    
    # Simple evaluation
    retrieval_indicators = [
        "retrieved information" in wiki_page.lower(),
        "cast" in wiki_page.lower() and "amitabh" in wiki_page.lower(),
        "director" in wiki_page.lower() and "yash chopra" in wiki_page.lower(),
        "1976" in wiki_page
    ]
    
    print(f"\nRetrieval Effectiveness: {sum(retrieval_indicators)}/4 indicators present")

Starting FLARE-Direct generation for 'Kabhie Kabhie'...

--- Iteration 1 ---
Generated: Error calling Llama: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout...
No uncertainty detected, continuing generation...

--- Iteration 2 ---
Generated: Kabhi Kabhie is a 1976 Indian musical drama film directed by Yash Chopra and written by Amitabh Bach...
No uncertainty detected, continuing generation...

--- Iteration 3 ---
Generated: Kabhi Kabhie: A 1976 Indian Musical Drama Film

**Plot**

Kabhi Kabhie is a nostalgic and poignant t...
No uncertainty detected, continuing generation...

FINAL WIKIPEDIA PAGE:
Error calling Llama: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=30)

Kabhi Kabhie is a 1976 Indian musical drama film directed by Yash Chopra and written by Amitabh Bachchan. The film stars Amitabh Bachchan, Rakhee Gulzar, Shashi Kapoor, Waheeda Rehman, and Neetu Singh.

**Plot**

The film tells the story of two friends, Vic